# Connections Solver Notebook

Author: Eric Nunes

This notebook is supposed to act as a "playground" for me to experiment with different procedures, algorithms, approaches, whatever.

## Load Dataset

I built a bot to collect the full game archive from the New York Times (all previous game data is stored in one of their APIs). I'm pulling the dataset from Kaggle, but a mirror exists on HuggingFace.

Source:https://www.kaggle.com/datasets/eric27n/the-new-york-times-connections

In [2]:
import numpy as np
import pandas as pd

In [11]:
df = pd.read_csv("hf://datasets/eric27n/NYT-Connections/Connections_Data.csv")
df['Word'] = df['Word'].fillna("NA")
df['Word'] = df['Word'].str.lower()
df['Group Name'] = df['Group Name'].str.lower()
grouped = df.groupby('Game ID')
result = []

for game_id, group in grouped:
  words = group['Word'].tolist()
  group_by_name = group.groupby('Group Name')
  solution = []
  
  for group_name, sub_group in group_by_name:
    group_words = sub_group['Word'].tolist()
    reason = sub_group['Group Name'].iloc[0]
    solution.append({'words': group_words, 'reason': reason})

  result.append({'words': words, 'solution': {'groups': solution}})

ds = result
ds_len = len(ds)
print(len(ds), ds[0])

508 {'words': ['snow', 'level', 'shift', 'kayak', 'heat', 'tab', 'bucks', 'return', 'jazz', 'hail', 'option', 'rain', 'sleet', 'racecar', 'mom', 'nets'], 'solution': {'groups': [{'words': ['shift', 'tab', 'return', 'option'], 'reason': 'keyboard keys'}, {'words': ['heat', 'bucks', 'jazz', 'nets'], 'reason': 'nba teams'}, {'words': ['level', 'kayak', 'racecar', 'mom'], 'reason': 'palindromes'}, {'words': ['snow', 'hail', 'rain', 'sleet'], 'reason': 'wet weather'}]}}


## Word2Vec

Note: One W2V model used is conceptnet-numberbatch. Download the English-only compressed text file from here: https://github.com/commonsense/conceptnet-numberbatch?tab=readme-ov-file

In [ ]:
import gensim.downloader as api
from gensim.models.word2vec import Word2Vec
from collections import Counter
import gzip
from gensim.models import KeyedVectors

gzipped_file_path = 'numberbatch-en-19.08.txt.gz'
decompressed_file_path = 'numberbatch-en-19.08.txt'

with gzip.open(gzipped_file_path, 'rt', encoding='utf-8') as f_in:
    with open(decompressed_file_path, 'w', encoding='utf-8') as f_out:
        f_out.write(f_in.read())

# Import different models
model_google = api.load('word2vec-google-news-300')
model_glove = api.load('glove-wiki-gigaword-300')
model_wiki = api.load('fasttext-wiki-news-subwords-300')
model_conceptnet = KeyedVectors.load_word2vec_format(decompressed_file_path, binary=False)

# Test: find similar words
print(f"GOOGLE NEWS: {model_google.most_similar('seattle')}")
print(f"GLOVE: {model_glove.most_similar('seattle')}")
print(f"WIKI: {model_wiki.most_similar('seattle')}")
print(f"CONCEPTNET': {model_conceptnet.most_similar('seattle')}")

Most similar words to 'seattle': [('university_of_washington', 0.9806408286094666), ('space_needle', 0.9797334671020508), ('seattleite', 0.9641170501708984), ('emerald_city', 0.9455490112304688), ('tacoma', 0.7643471360206604), ('spokane', 0.7531239986419678), ('portland', 0.7523225545883179), ('lake_chelan', 0.7268684506416321), ('washington', 0.7256752252578735), ('kennewick', 0.7251202464103699)]


In [29]:
# Extract words from ds[i]['words']
def guess(model, words):
  similarity_matrix = np.zeros((len(words), len(words)))
  for i, word1 in enumerate(words):
      for j, word2 in enumerate(words):
          if word1 in model and word2 in model:
              similarity_matrix[i, j] = model.similarity(word1, word2)
          else:
              similarity_matrix[i, j] = 0

  similarity_df = pd.DataFrame(similarity_matrix, index=words, columns=words)
  _max = 0
  argmax = 0
  argword = ""
  for idx, word in enumerate(words):
      similar_words = similarity_df[word].sort_values(ascending=False)
      if similar_words.iloc[1] > _max:
        _max = similar_words.iloc[1]
        argmax = idx
        argword = similar_words.index[1]

  build_list = [words[argmax], argword]

  words_copy = words.copy()
  for test_word in build_list:
    words_copy.remove(test_word)

  sim_list = []
  for test_word in words_copy:
    similarities = []
    for train_word in build_list:
        if train_word in model and test_word in model:
            similarity = model.similarity(train_word, test_word)
            similarities.append(similarity)
        else:
            similarities.append(0)  # Handle words not in the model
    average_similarity = sum(similarities) / len(similarities)
    sim_list.append(average_similarity)

  index_of_highest_value = sim_list.index(max(sim_list))
  build_list.append(words_copy[index_of_highest_value])

  words_copy = words.copy()
  for test_word in build_list:
    words_copy.remove(test_word)

  sim_list = []
  for test_word in words_copy:
    similarities = []
    for train_word in build_list:
        if train_word in model and test_word in model:
            similarity = model.similarity(train_word, test_word)
            similarities.append(similarity)
        else:
            similarities.append(0)  # Handle words not in the model
    average_similarity = sum(similarities) / len(similarities)
    sim_list.append(average_similarity)

  index_of_highest_value = sim_list.index(max(sim_list))
  build_list.append(words_copy[index_of_highest_value])

  return build_list

In [30]:
def eval_round(guess_list, solution):
  right_count = [0, 0, 0, 0]
  for final_word in guess_list:
    for idx, group in enumerate(solution['groups']):
      if final_word in group['words']:
        right_count[idx] += 1
  return max(right_count)

In [33]:
models = [model_google, model_glove, model_wiki, model_conceptnet]
model_names = ["Google News", "Glove", "Wikipedia", "ConceptNet"]
correct_idx = []
for idx, model in enumerate(models):
  print(f"======== {model_names[idx]} ========")
  right_list = []
  one_away_when = []
  for i in range(ds_len):
    if i == 300:
      continue
    guess_list = guess(model, ds[i]['words'])
    score = eval_round(guess_list, ds[i]['solution'])
    right_list.append(score)
    if score == 4 and i not in correct_idx:
      correct_idx.append(i)

  print(f"AVERAGE SCORE: {sum(right_list) / len(right_list)}")
  print(f"{model_names[idx]}: First Guess Correctness Distribution")
  for i in range(1, 5):
    print(f"{i}: {right_list.count(i)}")
print(f"Number of Games with At Least One Good First Guess: {len(correct_idx)}")

======== Google News ========
AVERAGE SCORE: 2.9664694280078896
Google News: First Guess Correctness Distribution
1: 10
2: 134
3: 226
4: 137
======== Glove ========
AVERAGE SCORE: 2.8599605522682445
Glove: First Guess Correctness Distribution
1: 12
2: 173
3: 196
4: 126
======== Wikipedia ========
AVERAGE SCORE: 2.9881656804733727
Wikipedia: First Guess Correctness Distribution
1: 13
2: 139
3: 196
4: 159
======== ConceptNet ========
AVERAGE SCORE: 3.138067061143984
ConceptNet: First Guess Correctness Distribution
1: 13
2: 95
3: 208
4: 191
Number of Games with At Least One Good First Guess: 285


## Transformers (BERT)

In [ ]:
from transformers import BertTokenizer, BertModel
import torch
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import normalize

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

In [ ]:
def get_word_embedding(word):
    inputs = tokenizer(word, return_tensors='pt')
    with torch.no_grad():
        outputs = model(**inputs)
    embeddings = outputs.last_hidden_state
    mean_embedding = embeddings.mean(dim=1).squeeze(0)
    return mean_embedding

In [38]:
def calculate_similarity(embedding1, embedding2):
    return cosine_similarity(embedding1.reshape(1, -1), embedding2.reshape(1, -1))[0][0]

def find_most_similar_pair(word_embeddings):
    max_similarity = -1
    most_similar_pair = (None, None)
    words = list(word_embeddings.keys())

    for i in range(len(words)):
        for j in range(i + 1, len(words)):
            sim = calculate_similarity(word_embeddings[words[i]], word_embeddings[words[j]])
            if sim > max_similarity:
                max_similarity = sim
                most_similar_pair = (words[i], words[j])

    return most_similar_pair

def expand_group(selected_words, remaining_words, word_embeddings):
    max_similarity = -1
    best_word = None

    for word in remaining_words:
        # Calculate average similarity between the current word and the selected words
        similarities = [calculate_similarity(word_embeddings[word], word_embeddings[sw]) for sw in selected_words]
        avg_similarity = np.mean(similarities)

        if avg_similarity > max_similarity:
            max_similarity = avg_similarity
            best_word = word

    return best_word

In [ ]:
def bert_guess(words):
    # Generate embeddings for each word
    word_embeddings = {word: get_word_embedding(word) for word in words}

    # Step 3: Find the pair of two words that are most similar to each other
    word1, word2 = find_most_similar_pair(word_embeddings)
    selected_words = [word1, word2]

    # Step 4-5: Find the next most similar word to the current selection
    remaining_words = [w for w in words if w not in selected_words]
    for _ in range(2):  # Repeat twice to find 3rd and 4th words
        best_word = expand_group(selected_words, remaining_words, word_embeddings)
        selected_words.append(best_word)
        remaining_words.remove(best_word)

    return selected_words

def eval_round(words, solution):
    right_count = [0, 0, 0, 0]
    for final_word in words:
        for idx, group in enumerate(solution['groups']):
            if final_word in group['words']:
                right_count[idx] += 1
    return max(right_count)

In [ ]:
right_list = []
for i in range(ds_len):
    if i > 0 and i % 100 == 0:
        print(f"Game {i}")
    if i == 300:
        continue
    words = ds[i]['words']
    soln = ds[i]['solution']
    optimal_guess = bert_guess(words)
    score = eval_round(optimal_guess, soln)
    right_list.append(score)

print(f"AVERAGE SCORE: {sum(right_list) / len(right_list)}")
print("========GAMES BY MAX NUM RIGHT ENTRIES========")
for i in range(1, 5):
    print(f"{i}: {right_list.count(i)}")

Game 100
Game 200
Game 300
Game 400
Game 500
AVERAGE SCORE: 2.222879684418146
========GAMES BY MAX NUM RIGHT ENTRIES========
1: 38
2: 332
3: 123
4: 14


...work in progress?